In [1]:
import pickle

import pandas as pd
import numpy as np

from copy import deepcopy

from tqdm.auto import tqdm
from time import time
from datetime import datetime

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error




from app.preprocessing import preprocess_data
from app.basepiplines import base_pipelines

from service.encodedata import EncodeDataService
from service.nonedata import NoneDataService
from service.normalizedata import NormalizeDataService
from service.modelservice import ModelService
from service.outlierdata import OutlierDataService

from processing.normalizedata import NormilizeData

from models.basemodel import Model

from config import TARGET_PARAM
from config import TRAIN_SCV, TEST_CSV, DATA_DESCRIPTION

from description_parser import parse_data_description


In [2]:
df_train = pd.read_csv(TRAIN_SCV, sep = ',')
df_test = pd.read_csv(TEST_CSV, sep = ',')

In [3]:
# Время выполнения этапа
time_table = {}

In [4]:
start = time()


In [5]:
# Список вариантов обработки данных, содержащий словари, состоящий из df_train, df_test и preprocessing_description
base_pipelines = []

In [6]:
#######################
# Базовый случай без изменений
base_pipeline = {"df_test": df_test,
                 "df_train": df_train,
                 "description": "Base: Original datasets."
                 }

base_pipelines.append(base_pipeline)

In [7]:
#######################
# Удалим строки с пропусками в train
df_train2 = deepcopy(df_train.dropna(axis=0))
df_test2 = deepcopy(df_test)

base_pipeline2 = {"df_test": df_test2,
                 "df_train": df_train2,
                 "description": "Base: Train dropna lines."
                 }
base_pipelines.append(base_pipeline2)

In [8]:
#######################
# Заполним пустые значения из файла description
cat_features_values = parse_data_description(DATA_DESCRIPTION)

df_train3 = deepcopy(df_train)
df_test3 = deepcopy(df_test)

df_train3 = df_train3.astype({'MSSubClass': str})
df_test3 = df_test3.astype({'MSSubClass': str})

cat_features = df_train3.select_dtypes(include='object').columns

for param in cat_features:
    for value in cat_features_values[param]:
        if (type(value) != int) and "NA=" in value:

            df_train3[param].fillna(value=value, inplace=True)

            df_test3[param].fillna(value=value, inplace=True)
base_pipeline3 = {"df_test": df_test3,
                 "df_train": df_train3,
                 "description": "Base: NA from description."
                 }
base_pipelines.append(base_pipeline3)

In [9]:
###############
# Проверим экспертную оценку: Согласно экспертному мнению риэлторов цена зависит от нескольких факторов:

# район застройки (Neighborhood, Condition1, Condition2, Street);
# вид жилья — дом, квартира, таунхаус, вилла (BldgType, HouseStyle, OverallQual);
# возраст недвижимости(YearBuilt);
# площадь помещения(GrLivArea).
expert_features = ['Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'YearBuilt', 'GrLivArea', 'OverallQual']

df_train4 = deepcopy(df_train)
df_test4 = deepcopy(df_test)

df_train4 = df_train4[expert_features]
df_train4['SalePrice'] = df_train['SalePrice']

df_test4 = df_test4[expert_features]

base_pipeline = {"df_test": df_test4,
                 "df_train": df_train4,
                 "description": "Base: expert features."
                 }

base_pipelines.append(base_pipeline)

In [10]:
#######################
# 5
correlated_params_to_del = ["GarageYrBlt", "TotRmsAbvGrd", "GarageCars", "1stFlrSF"]

df_train5 = deepcopy(df_train)
df_test5 = deepcopy(df_test)

df_train5.drop(columns=correlated_params_to_del, inplace=True)
df_test5.drop(columns=correlated_params_to_del, inplace=True)

# Removing features with almost 100% of the same value
non_uniq_val_params_to_del = ['Street', 'Utilities', 'Condition2', 'RoofMatl', 'Heating', 'LowQualFinSF',
                              '3SsnPorch', 'PoolArea', 'MiscVal', "SaleType", "Electrical", "Functional", 
                              'BsmtFinSF1', 'BsmtFinSF2']

df_train5.drop(columns=non_uniq_val_params_to_del, inplace=True)
df_test5.drop(columns=non_uniq_val_params_to_del, inplace=True)

# Removing features with a lot of missing values
missing_data_params_to_del = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'LotFrontage']

df_train5.drop(columns=missing_data_params_to_del, inplace=True)
df_test5.drop(columns=missing_data_params_to_del, inplace=True)


# Converting categorical to numerical
cat_col = ['ExterQual','ExterCond','BsmtQual', 'BsmtCond','HeatingQC','KitchenQual',
           'GarageQual','GarageCond', 'BsmtFinType2','BsmtFinType1', 'BsmtExposure', 'GarageFinish']

cat_feature_map = {'Ex': 5,'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA':0,
                   'GLQ': 6,'ALQ': 5,'BLQ': 4,'Rec': 3,'LwQ': 2,'Unf': 1, 'NA': 0,
                   'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'NA': 0,
                   'GdPrv': 4,'MnPrv': 3,'GdWo': 2, 'MnWw': 1, 'Fin': 4, 'RFn': 2, 'Unf': 1, 'NA': 0}

for col in cat_col:
    df_train5[col] = df_train5[col].map(cat_feature_map)
    df_test5[col] = df_test5[col].map(cat_feature_map)

# Преобразуем MSSubClass к str     
df_train5['MSSubClass'] = df_train5['MSSubClass'].apply(str)
df_test5['MSSubClass'] = df_test5['MSSubClass'].apply(str)

base_pipeline = {"df_test": df_test5,
                 "df_train": df_train5,
                 "description": "Base: expert processing."
                 }

base_pipelines.append(base_pipeline)

In [11]:
time_table["make base_pipeline"] = (round(time() - start), len(base_pipelines))


In [12]:
def preprocess_data(base_pipelines, action):
    pipelines = deepcopy(base_pipelines)
      
    pipelines = action(pipelines).process()
    print(action)
    NoneDescr = [pipeline for pipeline in pipelines if not pipeline.get("description")]
    print("NoneDescr", NoneDescr)

    return pipelines

In [13]:
start = time()

action = NoneDataService
print(f"Предобработка данных: {action}")
nonedata_pipelines = preprocess_data(base_pipelines, action)

time_table["process nonedata_pipelines"] = (round(time() - start), len(nonedata_pipelines))

Предобработка данных: <class 'service.nonedata.NoneDataService'>


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

<class 'service.nonedata.NoneDataService'>
NoneDescr []


In [14]:
start = time()

action = OutlierDataService
print(f"Предобработка данных: {action}")
outlierdata_pipelines = preprocess_data(nonedata_pipelines, action)

time_table["process outlierdata_pipelines"] = (round(time() - start), len(outlierdata_pipelines))

Предобработка данных: <class 'service.outlierdata.OutlierDataService'>


  0%|          | 0/30 [00:00<?, ?it/s]

<class 'service.outlierdata.OutlierDataService'>
NoneDescr []


In [15]:
start = time()

action = EncodeDataService

print(f"Предобработка данных: {action}")

pipelines_encoded = preprocess_data(outlierdata_pipelines, action)

time_table["process pipelines_encoded"] = (round(time() - start), len(pipelines_encoded))

Предобработка данных: <class 'service.encodedata.EncodeDataService'>


  0%|          | 0/60 [00:00<?, ?it/s]

<class 'service.encodedata.EncodeDataService'>
NoneDescr []


In [16]:
def normalize(pipelines):
    
    new_pipelines = []
    
    methods = [attribute for attribute in dir(NormilizeData) if callable(getattr(NormilizeData, attribute)) and attribute.startswith('__') is False and attribute.startswith('_') is False]

    for pipeline in tqdm(pipelines):
        for method in tqdm(methods):
            new_pipeline = deepcopy(pipeline)

            try:
                new_pipeline["df_train"], new_pipeline["df_test"] = NormilizeData(new_pipeline["df_train"], new_pipeline["df_test"])._get_method(method)
                new_pipeline["description"] = f"{pipeline.get('description')} NormilizeData: {method}."
                new_pipelines.append(new_pipeline)
            except Exception as e:
                new_pipeline["error"] = e
                print(method)
                print("description before", pipeline.get('description'))
                print(new_pipeline["error"])
                print("description after", new_pipeline.get('description'))
    
    return new_pipelines

In [17]:
start = time()

pipelines_normalized = normalize(pipelines_encoded)

time_table["process pipelines_normalized"] = (round(time() - start), len(pipelines_normalized))


  0%|          | 0/142 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [18]:
def modeling(pipelines, model):
    test_sizes = [0.05, 0.15, 0.2, 0.25]
    random_states = [12, 500, 1000]

    results = []

    for pipeline in tqdm(deepcopy(pipelines)):
        for test_size in test_sizes:
            for random_state in random_states:
                new_pipeline = deepcopy(pipeline)



                try:
                    x = new_pipeline.get('df_train').drop(columns=[TARGET_PARAM])
                    y = new_pipeline.get('df_train')[TARGET_PARAM]
                    X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=test_size, random_state=random_state)

                    model.fit(X_train, y_train)
                    
                    new_pipeline["y_pred_test"] = model.predict(new_pipeline.get("df_test")).round()

                    new_pipeline["y_pred_val"] = model.predict(X_val)
                    
                    scores = {}
                    scores['root_mean_squared_error']  = mean_squared_error(y_val, new_pipeline["y_pred_val"], squared=False)
                    scores['mean_squared_error'] = mean_squared_error(y_val, new_pipeline["y_pred_val"])
                    scores['mean_absolute_error'] = mean_absolute_error(y_val, new_pipeline["y_pred_val"])
                    scores['mean_absolute_percentage_error'] = mean_absolute_percentage_error(y_val, new_pipeline["y_pred_val"])

                    new_pipeline["scores"] = scores
                    new_pipeline["description"] = f"{new_pipeline['description']} test_size: {test_size}. random_state: {random_state}."
                    
                    if "coef_" in dir(model):
                        new_pipeline["feature_importance_"] = sorted(zip(X_train.columns, model.coef_), key=lambda x: -x[1])
                    elif "feature_importances_" in dir(model):
                        new_pipeline["feature_importance_"] = sorted(zip(X_train.columns, model.feature_importance_), key=lambda x: -x[1])
                    
                    results.append(new_pipeline)
                    
                except Exception as e:
                    new_pipeline["error"] = f"{e} test_size: {test_size}. random_state: {random_state}"
   
    return results


In [19]:
def scores_to_df(results)-> pd.DataFrame:
    score_dict = {}
    for n, result in enumerate(results):
        score_dict[n] = result["scores"]
    scores_df = pd.DataFrame(score_dict).T
    return scores_df 

In [20]:
def description_to_df(results)-> pd.DataFrame:
    description_dict = {}
    for n, result in enumerate(results):
        tmp_dict = {}
        tmp_list = result["description"].split(". ")
        
        for elem in tmp_list:
            key, value = elem.split(": ") 
            tmp_dict[key] = value.rstrip(".")
            
        description_dict[n] = tmp_dict
        
    description_df = pd.DataFrame(description_dict).T
    return description_df 

In [21]:
def get_mins(results):
    df_scores = scores_to_df(results)
    df_scores.index.rename('ID', inplace= True )
    
    df_descr = description_to_df(results)
    df_descr.index.rename('ID', inplace= True )
    
    df_scores = df_scores.merge(df_descr, left_index=True, right_index=True)
    
    df_min = df_scores.loc[[df_scores["root_mean_squared_error"].idxmin(),
                            df_scores["mean_absolute_error"].idxmin(),
                            df_scores["mean_absolute_percentage_error"].idxmin()]].drop_duplicates()
    
    return df_scores, df_min.round(2)

In [22]:
min_results = []

In [23]:
# LinearRegression
from sklearn.linear_model import LinearRegression

start = time()

LinearRegression_results = modeling(pipelines_normalized, LinearRegression())

time_table["LinearRegression"] = (round(time() - start), len(LinearRegression_results))


  0%|          | 0/520 [00:00<?, ?it/s]

In [36]:
lr_scores, lr_min = get_mins(LinearRegression_results)
lr_min["Model"] = "LinearRegression"
min_results.append(lr_min)
lr_min

,root_mean_squared_error,mean_squared_error,mean_absolute_error,mean_absolute_percentage_error,Base,NoneData,OutlierData,EncodeData,NormilizeData,test_size,random_state,Model
ID,,,,,,,,,,,,
1320,19603.17,3.842844e+08,14517.18,0.08,Original datasets,fillna_with_train_most_common_and_mean,remove_target_outliers,target_encode,standart_scaler_on_all_data,0.05,12,LinearRegression
720,19710.24,3.884935e+08,14488.19,0.08,Original datasets,fillna_with_all_data_most_common_and_mean,remove_target_outliers,target_encode,min_max_scaler_on_all_data,0.05,12,LinearRegression
1596,19728.90,3.892294e+08,14507.26,0.08,Original datasets,fillna_with_train_most_common_and_median,remove_target_outliers,target_encode,min_max_scaler_on_train,0.05,12,LinearRegression


In [77]:
def get_feature_importance_top_5(results):
    fi_dict = {}
    for result in results:

        count = 0

        for feature, num in result["feature_importance_"]:
            fi_dict[feature] = fi_dict.get(feature, 0) + 1
            count += 1
            if count == 5:
                break
    return fi_dict
        
get_feature_importance_top_5(LinearRegression_results)

{'1stFlrSF': 1462,
 'GrLivArea': 4352,
 'OverallQual': 4328,
 'LotArea': 574,
 'YearBuilt': 2284,
 'PoolArea': 439,
 'OverallCond': 876,
 'TotRmsAbvGrd': 48,
 '2ndFlrSF': 1300,
 'LowQualFinSF': 953,
 'RoofMatl': 93,
 'Neighborhood': 2738,
 'ExterQual': 128,
 'WoodDeckSF': 26,
 'Fireplaces': 91,
 'TotalBsmtSF': 1461,
 'BsmtFinSF1': 1100,
 'BsmtUnfSF': 1092,
 'BsmtFinSF2': 855,
 'Utilities': 408,
 'GarageCars': 195,
 'PoolQC': 203,
 'KitchenQual': 208,
 'Exterior1st': 162,
 'BsmtQual': 168,
 'MiscVal': 6,
 'MasVnrArea': 178,
 'Street': 12,
 'BsmtFullBath': 130,
 'ScreenPorch': 6,
 'Condition2': 240,
 'Condition1': 972,
 'BldgType': 564,
 'SaleCondition': 26,
 'LandSlope': 3,
 'MSSubClass': 2,
 'GarageArea': 457,
 'BsmtExposure': 110,
 'FullBath': 11,
 'GarageQual': 55,
 'BsmtFinType1': 4}

In [78]:
lr_fi = get_feature_importance_top_5(LinearRegression_results)
lr_fi_list = [(param, value) for param, value in lr_fi.items()]
sorted(lr_fi_list, key=(lambda x: x[-1]))
    
    

[('MSSubClass', 2),
 ('LandSlope', 3),
 ('BsmtFinType1', 4),
 ('MiscVal', 6),
 ('ScreenPorch', 6),
 ('FullBath', 11),
 ('Street', 12),
 ('WoodDeckSF', 26),
 ('SaleCondition', 26),
 ('TotRmsAbvGrd', 48),
 ('GarageQual', 55),
 ('Fireplaces', 91),
 ('RoofMatl', 93),
 ('BsmtExposure', 110),
 ('ExterQual', 128),
 ('BsmtFullBath', 130),
 ('Exterior1st', 162),
 ('BsmtQual', 168),
 ('MasVnrArea', 178),
 ('GarageCars', 195),
 ('PoolQC', 203),
 ('KitchenQual', 208),
 ('Condition2', 240),
 ('Utilities', 408),
 ('PoolArea', 439),
 ('GarageArea', 457),
 ('BldgType', 564),
 ('LotArea', 574),
 ('BsmtFinSF2', 855),
 ('OverallCond', 876),
 ('LowQualFinSF', 953),
 ('Condition1', 972),
 ('BsmtUnfSF', 1092),
 ('BsmtFinSF1', 1100),
 ('2ndFlrSF', 1300),
 ('TotalBsmtSF', 1461),
 ('1stFlrSF', 1462),
 ('YearBuilt', 2284),
 ('Neighborhood', 2738),
 ('OverallQual', 4328),
 ('GrLivArea', 4352)]

5664

In [65]:
def get_fi(results):
    df_scores = scores_to_df(results)
    df_scores.index.rename('ID', inplace= True )
    
    df_descr = description_to_df(results)
    df_descr.index.rename('ID', inplace= True )
    
    df_scores = df_scores.merge(df_descr, left_index=True, right_index=True)
    
    df_min = df_scores.loc[[df_scores["root_mean_squared_error"].idxmin(),
                            df_scores["mean_absolute_error"].idxmin(),
                            df_scores["mean_absolute_percentage_error"].idxmin()]].drop_duplicates()
    
    

[{'1stFlrSF': 194535.42989919998,
  'GrLivArea': 187425.84872248428,
  'OverallQual': 135239.91035113388,
  'LotArea': 103138.10857890871,
  'YearBuilt': 66934.2145707325},
 {'1stFlrSF': 172223.36818223604,
  'GrLivArea': 163974.59425232763,
  'OverallQual': 143095.2176212339,
  'LotArea': 85389.23196861049,
  'YearBuilt': 64496.41294598826},
 {'1stFlrSF': 168000.80836672266,
  'GrLivArea': 160812.04656904275,
  'OverallQual': 139393.43728719684,
  'LotArea': 95273.16202740063,
  'YearBuilt': 66006.31469086492},
 {'1stFlrSF': 191891.39334538844,
  'GrLivArea': 186543.09338261132,
  'OverallQual': 143397.66161861646,
  'LotArea': 100242.49946170404,
  'YearBuilt': 62825.577362050855},
 {'1stFlrSF': 164639.5283541764,
  'GrLivArea': 157488.04628134507,
  'OverallQual': 150512.9530251026,
  'LotArea': 88262.34892850916,
  'YearBuilt': 62731.49100135038},
 {'1stFlrSF': 163140.18762093014,
  'GrLivArea': 157470.37273881718,
  'OverallQual': 146551.2312404628,
  'LotArea': 86513.92388449797,

In [80]:
# KNN
from sklearn.neighbors import KNeighborsRegressor

start = time()

KNeighborsRegressor_results = modeling(pipelines_normalized, KNeighborsRegressor())

time_table["KNeighborsRegressor"] = (round(time() - start), len(KNeighborsRegressor_results))


  0%|          | 0/520 [00:00<?, ?it/s]

In [37]:
knn_scores, knn_min = get_mins(KNeighborsRegressor_results)
knn_min["Model"] = "KNeighborsRegressor"
min_results.append(knn_min)
knn_min

,root_mean_squared_error,mean_squared_error,mean_absolute_error,mean_absolute_percentage_error,Base,NoneData,OutlierData,EncodeData,NormilizeData,test_size,random_state,Model
ID,,,,,,,,,,,,
2810,23876.09,5.700676e+08,16127.29,0.11,expert features,dropna_columns,do_not_remove_outliers,label_encode_on_all_data,standart_scaler_on_all_data,0.05,1000,KNeighborsRegressor
2821,27894.04,7.780776e+08,16584.69,0.09,expert features,dropna_columns,do_not_remove_outliers,label_encode_on_all_data,standart_scaler_on_train,0.05,500,KNeighborsRegressor


In [79]:
knn_fi = get_feature_importance_top_5(KNeighborsRegressor_results)
knn_fi_list = [(param, value) for param, value in knn_fi.items()]
sorted(knn_fi_list, key=(lambda x: x[-1]))


KeyError: 'feature_importance_'

In [27]:
import lightgbm as lgb
LGB_model = lgb.LGBMRegressor(verbose=0)

start = time()

LGBMRegressor_results = modeling(pipelines_normalized, LGB_model)

time_table["LGBMRegressor"] = (round(time() - start), len(LGBMRegressor_results))



  0%|          | 0/520 [00:00<?, ?it/s]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000982 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000869 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000998 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001035 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001012 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001106 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001148 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001142 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001348 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001020 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001144 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001010 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001152 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001094 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001102 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001099 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001007 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001152 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001160 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001070 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001119 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001157 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001161 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001078 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM]

...

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001814 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001785 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001879 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001637 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001673 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001647 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001650 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001622 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001803 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002014 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002049 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001951 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002030 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001959 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001782 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001812 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001947 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001759 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001591 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001974 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001587 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001544 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001494 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001479 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001618 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001492 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001545 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001537 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001686 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001585 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM]

In [38]:
# lgbm_scores, lgbm_min = get_mins(LGBMRegressor_results)
# lgbm_min["Model"] = "LGBMRegressor"
# min_results.append(lgbm_min)
# lgbm_min

KeyError: 'root_mean_squared_error'

In [29]:
from sklearn.tree import DecisionTreeRegressor
DTR = DecisionTreeRegressor(max_depth=5)

start = time()

DecisionTreeRegressor_results = modeling(pipelines_normalized, DTR)

time_table["DecisionTreeRegressor"] = (round(time() - start), len(DecisionTreeRegressor_results))


  0%|          | 0/520 [00:00<?, ?it/s]

In [39]:
# dtr_scores, dtr_min = get_mins(DecisionTreeRegressor_results)
# dtr_min["Model"] = "DecisionTreeRegressor"
# min_results.append(dtr_min)
# dtr_min

KeyError: 'root_mean_squared_error'

In [31]:
from sklearn.ensemble import RandomForestRegressor
RFR = RandomForestRegressor()

start = time()

RandomForestRegressor_results = modeling(pipelines_normalized, RFR)

time_table["RandomForestRegressor"] = (round(time() - start), len(RandomForestRegressor_results))


  0%|          | 0/520 [00:00<?, ?it/s]

In [40]:
# rf_scores, rf_min = get_mins(RandomForestRegressor_results)
# rf_min["Model"] = "RandomForestRegressor"
# min_results.append(rf_min)
# rf_min

KeyError: 'root_mean_squared_error'

In [33]:
import xgboost as XGB
XGB_model = XGB.XGBRegressor()


start = time()

XGBRegressor_results = modeling(pipelines_normalized, XGB_model)

time_table["XGBRegressor"] = (round(time() - start), len(XGBRegressor_results))



  0%|          | 0/520 [00:00<?, ?it/s]

In [41]:
# xgb_scores, xgb_min = get_mins(XGBRegressor_results)
# xgb_min["Model"] = "XGBRegressor"
# min_results.append(xgb_min)
# xgb_min

KeyError: 'root_mean_squared_error'

In [35]:
# import catboost as cb
# CB_model = cb.CatBoostRegressor(verbose=0)

# start = time()

# CatBoostRegressor_results = modeling(pipelines_normalized, CB_model)

# time_table["CatBoostRegressor"] = (round(time() - start), len(CatBoostRegressor_results))



  0%|          | 0/520 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# cb_scores, cb_min = get_mins(CatBoostRegressor_results)
# cb_min["Model"] = "CatBoostRegressor"
# min_results.append(cb_min)
# cb_min

In [42]:
score_table = pd.concat(min_results)
score_table

,root_mean_squared_error,mean_squared_error,mean_absolute_error,mean_absolute_percentage_error,Base,NoneData,OutlierData,EncodeData,NormilizeData,test_size,random_state,Model
ID,,,,,,,,,,,,
1320,19603.17,3.842844e+08,14517.18,0.08,Original datasets,fillna_with_train_most_common_and_mean,remove_target_outliers,target_encode,standart_scaler_on_all_data,0.05,12,LinearRegression
720,19710.24,3.884935e+08,14488.19,0.08,Original datasets,fillna_with_all_data_most_common_and_mean,remove_target_outliers,target_encode,min_max_scaler_on_all_data,0.05,12,LinearRegression
1596,19728.90,3.892294e+08,14507.26,0.08,Original datasets,fillna_with_train_most_common_and_median,remove_target_outliers,target_encode,min_max_scaler_on_train,0.05,12,LinearRegression
2810,23876.09,5.700676e+08,16127.29,0.11,expert features,dropna_columns,do_not_remove_outliers,label_encode_on_all_data,standart_scaler_on_all_data,0.05,1000,KNeighborsRegressor
2821,27894.04,7.780776e+08,16584.69,0.09,expert features,dropna_columns,do_not_remove_outliers,label_encode_on_all_data,standart_scaler_on_train,0.05,500,KNeighborsRegressor


In [ ]:
score_table.sort_values(by='root_mean_squared_error')['root_mean_squared_error', 'mean_squared_error', 'mean_absolute_error',
       'mean_absolute_percentage_error', 'NoneData', 'EncodeData',
       'NormilizeData', 'test_size', 'random_state', 'Model']

In [ ]:
score_table.columns

In [43]:
time_table

{'make base_pipeline': (0, 5),
 'process nonedata_pipelines': (2, 30),
 'process outlierdata_pipelines': (1, 60),
 'process pipelines_encoded': (23, 142),
 'process pipelines_normalized': (27, 520),
 'LinearRegression': (73, 5664),
 'KNeighborsRegressor': (237, 5664),
 'LGBMRegressor': (664, 0),
 'DecisionTreeRegressor': (64, 0),
 'RandomForestRegressor': (6109, 0),
 'XGBRegressor': (1390, 0)}

In [ ]:
t1 = {'LinearRegression': (20, 1392),
 'KNeighborsRegressor': (98, 1392),
 'LGBMRegressor': (243, 1392),
 'DecisionTreeRegressor': (30, 1392),
 'RandomForestRegressor': (1871, 1392),
 'XGBRegressor': (372, 1392),
 'CatBoostRegressor': (15056, 1392)}
df_t1 = pd.DataFrame.from_dict(t1).T
df_t1.sort_values(by=0)


In [44]:
df_time = pd.DataFrame.from_dict(time_table).T
df_time.sort_values(by=0)

,0,1
make base_pipeline,0,5
process outlierdata_pipelines,1,60
process nonedata_pipelines,2,30
process pipelines_encoded,23,142
process pipelines_normalized,27,520
DecisionTreeRegressor,64,0
LinearRegression,73,5664
KNeighborsRegressor,237,5664
LGBMRegressor,664,0
XGBRegressor,1390,0


In [48]:
lr_scores

,root_mean_squared_error,mean_squared_error,mean_absolute_error,mean_absolute_percentage_error,Base,NoneData,OutlierData,EncodeData,NormilizeData,test_size,random_state
ID,,,,,,,,,,,
0,50091.355987,2.509144e+09,23171.610193,0.123589,Original datasets,dropna_columns,do_not_remove_outliers,label_encode_on_all_data,min_max_scaler_on_all_data,0.05,12
1,31645.229935,1.001421e+09,22312.081129,0.131284,Original datasets,dropna_columns,do_not_remove_outliers,label_encode_on_all_data,min_max_scaler_on_all_data,0.05,500
2,27655.351130,7.648184e+08,19979.048147,0.122378,Original datasets,dropna_columns,do_not_remove_outliers,label_encode_on_all_data,min_max_scaler_on_all_data,0.05,1000
3,39558.885515,1.564905e+09,22067.925054,0.125565,Original datasets,dropna_columns,do_not_remove_outliers,label_encode_on_all_data,min_max_scaler_on_all_data,0.15,12
4,27558.197574,7.594543e+08,20018.878849,0.124530,Original datasets,dropna_columns,do_not_remove_outliers,label_encode_on_all_data,min_max_scaler_on_all_data,0.15,500
...,...,...,...,...,...,...,...,...,...,...,...
5659,21254.021812,4.517334e+08,16155.478396,0.101679,expert processing,fillna_with_train_most_common_and_median,remove_target_outliers,target_encode,standart_scaler_on_train,0.2,500
5660,22470.962130,5.049441e+08,17035.614715,0.103985,expert processing,fillna_with_train_most_common_and_median,remove_target_outliers,target_encode,standart_scaler_on_train,0.2,1000
5661,34594.104232,1.196752e+09,15875.359999,0.101573,expert processing,fillna_with_train_most_common_and_median,remove_target_outliers,target_encode,standart_scaler_on_train,0.25,12


In [ ]:
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.model_selection import GridSearchCV
# DTR = DecisionTreeRegressor(max_depth=5)
# DTR.fit(X_train, y_train)

# print("TRAIN")
# y_pred = DTR.predict(X_train)
# print(f"RMSE:       {mean_squared_error(y_train, y_pred) ** 0.5}")
# print(f"MAPE:      {mean_absolute_percentage_error(y_train+1, y_pred+1)}")
# print("TEST")
# y_pred = DTR.predict(X_val)
# print(f"RMSE:       {mean_squared_error(y_val, y_pred) ** 0.5}")
# print(f"MAPE:      {mean_absolute_percentage_error(y_val+1, y_pred+1)}")

# grid_DTR = GridSearchCV(
#             DecisionTreeRegressor(),               
#             param_grid = {                          
#                 'max_depth': range(1, 8),
#                 'min_samples_leaf': range(1, 8),
#             },
#             scoring = 'neg_mean_absolute_error',  
#             cv = 3,                                 
#             n_jobs = -1,                            
#             return_train_score = True,              
#             verbose = 5,                            
#         )
# grid_DTR.fit(X_train, y_train)

# print("TRAIN")
# y_pred = grid_DTR.best_estimator_.predict(X_train)
# print(f"RMSE:       {mean_squared_error(y_train, y_pred) ** 0.5}")
# print(f"MAPE:      {mean_absolute_percentage_error(y_train+1, y_pred+1)}")
# print("TEST")
# y_pred_DTR = grid_DTR.best_estimator_.predict(X_val)
# print(f"RMSE:       {mean_squared_error(y_val, y_pred_DTR) ** 0.5}")
# print(f"MAPE:      {mean_absolute_percentage_error(y_val+1, y_pred_DTR+1)}")

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.ensemble import RandomForestRegressor
# RFR = RandomForestRegressor()
# RFR.fit(X_train, y_train)

# print("TRAIN")
# y_pred = RFR.predict(X_train)
# print(f"RMSE:       {mean_squared_error(y_train, y_pred) ** 0.5}")
# print(f"MAPE:      {mean_absolute_percentage_error(y_train+1, y_pred+1)}")
# print("TEST")
# y_pred = RFR.predict(X_val)
# print(f"RMSE:       {mean_squared_error(y_val, y_pred) ** 0.5}")
# print(f"MAPE:      {mean_absolute_percentage_error(y_val+1, y_pred+1)}")

# grid_RFR = RandomizedSearchCV(
#             RandomForestRegressor(),                
#             param_distributions = {                
#                 'max_depth': range(6, 20),
#                 'min_samples_split': range(2, 9),
#                 'max_features': range(2, 11)
#             },
#             scoring = 'neg_mean_absolute_error',    
#             cv = 3,                                 
#             n_jobs = -1,                            
#             return_train_score = True,              
#             n_iter = 16,                              
#             verbose = 5,                            
#         )

# grid_RFR.fit(X_train, y_train)

# print("TRAIN")
# y_pred = grid_RFR.best_estimator_.predict(X_train)
# print(f"RMSE:       {mean_squared_error(y_train, y_pred) ** 0.5}")
# print(f"MAPE:      {mean_absolute_percentage_error(y_train+1, y_pred+1)}")
# print("TEST")
# y_pred_RFR = grid_RFR.best_estimator_.predict(X_val)
# print(f"RMSE:       {mean_squared_error(y_val, y_pred_RFR) ** 0.5}")
# print(f"MAPE:      {mean_absolute_percentage_error(y_val+1, y_pred_RFR+1)}")

In [ ]:
# import lightgbm as lgb
# LGB_model = lgb.LGBMRegressor(verbose=0)
# LGB_model.fit(X_train, y_train)

# print("TRAIN")
# y_pred = LGB_model.predict(X_train)
# print(f"RMSE:       {mean_squared_error(y_train, y_pred) ** 0.5}")
# print(f"MAPE:      {mean_absolute_percentage_error(y_train+1, y_pred+1)}")
# print("TEST")
# y_pred = LGB_model.predict(X_val)
# print(f"RMSE:       {mean_squared_error(y_val, y_pred) ** 0.5}")
# print(f"MAPE:      {mean_absolute_percentage_error(y_val+1, y_pred+1)}")

# grid_LGB = RandomizedSearchCV(
#             lgb.LGBMRegressor(verbose=0),          
#             param_distributions = {                 
#                 'max_depth': range(2, 7),
#                 'lr': np.linspace(0.001, 0.3, 100),
#             },
#             scoring = 'neg_mean_absolute_error',    
#             cv = 3,                                 
#             n_jobs = -1,                            
#             return_train_score = True,              
#             n_iter=30,                              
#             verbose = 5,                            
#         )

# grid_LGB.fit(X_train, y_train)

# print("TRAIN")
# y_pred = grid_LGB.best_estimator_.predict(X_train)
# print(f"RMSE:       {mean_squared_error(y_train, y_pred) ** 0.5}")
# print(f"MAPE:      {mean_absolute_percentage_error(y_train+1, y_pred+1)}")
# print("TEST")
# y_pred_LGB = grid_LGB.best_estimator_.predict(X_val)
# print(f"RMSE:       {mean_squared_error(y_val, y_pred_LGB) ** 0.5}")
# print(f"MAPE:      {mean_absolute_percentage_error(y_val+1, y_pred_LGB+1)}")

In [ ]:
# import xgboost as XGB

# XGB_model = XGB.XGBRegressor()
# XGB_model.fit(X_train, y_train)

# print("TRAIN")
# y_pred = XGB_model.predict(X_train)
# print(f"RMSE:       {mean_squared_error(y_train, y_pred) ** 0.5}")
# print(f"MAPE:      {mean_absolute_percentage_error(y_train+1, y_pred+1)}")
# print("TEST")
# y_pred = XGB_model.predict(X_val)
# print(f"RMSE:       {mean_squared_error(y_val, y_pred) ** 0.5}")
# print(f"MAPE:      {mean_absolute_percentage_error(y_val+1, y_pred+1)}")

# grid_XGB = RandomizedSearchCV(
#             XGB.XGBRegressor(verbose=0),          #
#             param_distributions = {                 
#                 'max_depth': range(2, 7),
#                 'lr': np.linspace(0.001, 0.3, 100),
#             },
#             scoring = 'neg_mean_absolute_error',    
#             cv = 3,                                 
#             n_jobs = -1,                            
#             return_train_score = True,              
#             n_iter=30,                              
#             verbose = 5,                            
#         )

# grid_XGB.fit(X_train, y_train)

# print("TRAIN")
# y_pred = grid_XGB.best_estimator_.predict(X_train)
# print(f"RMSE:       {mean_squared_error(y_train, y_pred) ** 0.5}")
# print(f"MAPE:      {mean_absolute_percentage_error(y_train+1, y_pred+1)}")
# print("TEST")
# y_pred_XGB = grid_XGB.best_estimator_.predict(X_val)
# print(f"RMSE:       {mean_squared_error(y_val, y_pred_XGB) ** 0.5}")
# print(f"MAPE:      {mean_absolute_percentage_error(y_val+1, y_pred_XGB+1)}")

In [ ]:
# import catboost as cb

# CB_model = cb.CatBoostRegressor(verbose=0)
# CB_model.fit(X_train, y_train)

# print("TRAIN")
# y_pred = CB_model.predict(X_train)
# print(f"RMSE:       {mean_squared_error(y_train, y_pred) ** 0.5}")
# print(f"MAPE:      {mean_absolute_percentage_error(y_train+1, y_pred+1)}")
# print("TEST")
# y_pred = CB_model.predict(X_val)
# print(f"RMSE:       {mean_squared_error(y_val, y_pred) ** 0.5}")
# print(f"MAPE:      {mean_absolute_percentage_error(y_val+1, y_pred+1)}")

# grid_CB = RandomizedSearchCV(
#             cb.CatBoostRegressor(verbose=0),       
#             param_distributions = {                 
#                 'max_depth': range(2, 7),
#                 'learning_rate': np.linspace(0.001, 0.3, 100),
#             },
#             scoring = 'neg_mean_absolute_error',    
#             cv = 3,                                 
#             n_jobs = -1,                            
#             return_train_score = True,              
#             n_iter=10,                              
#             verbose = 5,                           
#         )
# grid_CB.fit(X_train, y_train)

# print("TRAIN")
# y_pred = grid_CB.best_estimator_.predict(X_train)
# print(f"RMSE:       {mean_squared_error(y_train, y_pred) ** 0.5}")
# print(f"MAPE:      {mean_absolute_percentage_error(y_train+1, y_pred+1)}")
# print("TEST")
# y_pred_CB = grid_CB.best_estimator_.predict(X_val)
# print(f"RMSE:       {mean_squared_error(y_val, y_pred_CB) ** 0.5}")
# print(f"MAPE:      {mean_absolute_percentage_error(y_val+1, y_pred_CB+1)}")